## Installation

In [1]:
!sudo apt-get install git -y
!git clone https://github.com/Zeta36/chess-alpha-zero
!pip install -r ./chess-alpha-zero/requirements.txt
!pip install gym_chess

Reading package lists... Done
Building dependency tree       
Reading state information... Done
git is already the newest version (1:2.7.4-0ubuntu1.10).
0 upgraded, 0 newly installed, 0 to remove and 28 not upgraded.
fatal: destination path 'chess-alpha-zero' already exists and is not an empty directory.
  Using cached https://files.pythonhosted.org/packages/9d/28/11d5d1f2f5c7e65c7d25e825f337c04b22a91d2021aa9a1eaae7c0d9d61d/python_chess-0.25.1-py3-none-any.whl
ERROR: gym-chess 0.1.1 has requirement python-chess<0.32.0,>=0.31.1, but you'll have python-chess 0.25.1 which is incompatible.
  Found existing installation: python-chess 0.31.4
    Uninstalling python-chess-0.31.4:
      Successfully uninstalled python-chess-0.31.4
  Using cached https://files.pythonhosted.org/packages/6e/40/6d3f685d3ca87d7ec88b1921f8f3625e79cae628629801186e3c76201260/python_chess-0.31.4-py3-none-any.whl
  Found existing installation: python-chess 0.25.1
    Uninstalling python-chess-0.25.1:
      Successfully 

In [2]:
import numpy as np
import random
import gym
import gym_chess
import chess
import chess.svg
import chess.polyglot

from datetime import datetime
import timeit
from io import BytesIO
from PIL import Image
#from cairosvg import svg2png
#import cv2
from IPython.display import clear_output
from IPython.display import SVG

# MiniMax

In [3]:
def evaluate_board():
    
    if board.is_checkmate():
        if board.turn:
            return -9999
        else:
            return 9999
    if board.is_stalemate():
        return 0
    if board.is_insufficient_material():
        return 0
    
    wp = len(board.pieces(chess.PAWN, chess.WHITE))
    bp = len(board.pieces(chess.PAWN, chess.BLACK))
    wn = len(board.pieces(chess.KNIGHT, chess.WHITE))
    bn = len(board.pieces(chess.KNIGHT, chess.BLACK))
    wb = len(board.pieces(chess.BISHOP, chess.WHITE))
    bb = len(board.pieces(chess.BISHOP, chess.BLACK))
    wr = len(board.pieces(chess.ROOK, chess.WHITE))
    br = len(board.pieces(chess.ROOK, chess.BLACK))
    wq = len(board.pieces(chess.QUEEN, chess.WHITE))
    bq = len(board.pieces(chess.QUEEN, chess.BLACK))
    
    material = 100*(wp-bp)+320*(wn-bn)+330*(wb-bb)+500*(wr-br)+900*(wq-bq)
    
    pawnsq = sum([pawntable[i] for i in board.pieces(chess.PAWN, chess.WHITE)])
    pawnsq= pawnsq + sum([-pawntable[chess.square_mirror(i)] 
                                    for i in board.pieces(chess.PAWN, chess.BLACK)])
    knightsq = sum([knightstable[i] for i in board.pieces(chess.KNIGHT, chess.WHITE)])
    knightsq = knightsq + sum([-knightstable[chess.square_mirror(i)] 
                                    for i in board.pieces(chess.KNIGHT, chess.BLACK)])
    bishopsq= sum([bishopstable[i] for i in board.pieces(chess.BISHOP, chess.WHITE)])
    bishopsq= bishopsq + sum([-bishopstable[chess.square_mirror(i)] 
                                    for i in board.pieces(chess.BISHOP, chess.BLACK)])
    rooksq = sum([rookstable[i] for i in board.pieces(chess.ROOK, chess.WHITE)]) 
    rooksq = rooksq + sum([-rookstable[chess.square_mirror(i)] 
                                    for i in board.pieces(chess.ROOK, chess.BLACK)])
    queensq = sum([queenstable[i] for i in board.pieces(chess.QUEEN, chess.WHITE)]) 
    queensq = queensq + sum([-queenstable[chess.square_mirror(i)] 
                                    for i in board.pieces(chess.QUEEN, chess.BLACK)])
    kingsq = sum([kingstable[i] for i in board.pieces(chess.KING, chess.WHITE)]) 
    kingsq = kingsq + sum([-kingstable[chess.square_mirror(i)] 
                                    for i in board.pieces(chess.KING, chess.BLACK)])
    
    eval = material + pawnsq + knightsq + bishopsq+ rooksq+ queensq + kingsq
    if board.turn:
        return eval
    else:
        return -eval

In [4]:
pawntable = [
 0,  0,  0,  0,  0,  0,  0,  0,
 5, 10, 10,-20,-20, 10, 10,  5,
 5, -5,-10,  0,  0,-10, -5,  5,
 0,  0,  0, 20, 20,  0,  0,  0,
 5,  5, 10, 25, 25, 10,  5,  5,
10, 10, 20, 30, 30, 20, 10, 10,
50, 50, 50, 50, 50, 50, 50, 50,
 0,  0,  0,  0,  0,  0,  0,  0]

knightstable = [
-50,-40,-30,-30,-30,-30,-40,-50,
-40,-20,  0,  5,  5,  0,-20,-40,
-30,  5, 10, 15, 15, 10,  5,-30,
-30,  0, 15, 20, 20, 15,  0,-30,
-30,  5, 15, 20, 20, 15,  5,-30,
-30,  0, 10, 15, 15, 10,  0,-30,
-40,-20,  0,  0,  0,  0,-20,-40,
-50,-40,-30,-30,-30,-30,-40,-50]

bishopstable = [
-20,-10,-10,-10,-10,-10,-10,-20,
-10,  5,  0,  0,  0,  0,  5,-10,
-10, 10, 10, 10, 10, 10, 10,-10,
-10,  0, 10, 10, 10, 10,  0,-10,
-10,  5,  5, 10, 10,  5,  5,-10,
-10,  0,  5, 10, 10,  5,  0,-10,
-10,  0,  0,  0,  0,  0,  0,-10,
-20,-10,-10,-10,-10,-10,-10,-20]

rookstable = [
  0,  0,  0,  5,  5,  0,  0,  0,
 -5,  0,  0,  0,  0,  0,  0, -5,
 -5,  0,  0,  0,  0,  0,  0, -5,
 -5,  0,  0,  0,  0,  0,  0, -5,
 -5,  0,  0,  0,  0,  0,  0, -5,
 -5,  0,  0,  0,  0,  0,  0, -5,
  5, 10, 10, 10, 10, 10, 10,  5,
 0,  0,  0,  0,  0,  0,  0,  0]

queenstable = [
-20,-10,-10, -5, -5,-10,-10,-20,
-10,  0,  0,  0,  0,  0,  0,-10,
-10,  5,  5,  5,  5,  5,  0,-10,
  0,  0,  5,  5,  5,  5,  0, -5,
 -5,  0,  5,  5,  5,  5,  0, -5,
-10,  0,  5,  5,  5,  5,  0,-10,
-10,  0,  0,  0,  0,  0,  0,-10,
-20,-10,-10, -5, -5,-10,-10,-20]

kingstable = [
 20, 30, 10,  0,  0, 10, 30, 20,
 20, 20,  0,  0,  0,  0, 20, 20,
-10,-20,-20,-20,-20,-20,-20,-10,
-20,-30,-30,-40,-40,-30,-30,-20,
-30,-40,-40,-50,-50,-40,-40,-30,
-30,-40,-40,-50,-50,-40,-40,-30,
-30,-40,-40,-50,-50,-40,-40,-30,
-30,-40,-40,-50,-50,-40,-40,-30]

In [5]:
def alphabeta( alpha, beta, depthleft ):
    bestscore = -9999
    if( depthleft == 0 ):
        return quiesce( alpha, beta )
    for move in board.legal_moves:
        board.push(move)   
        score = -alphabeta( -beta, -alpha, depthleft - 1 )
        board.pop()
        if( score >= beta ):
            return score
        if( score > bestscore ):
            bestscore = score
        if( score > alpha ):
            alpha = score   
    return bestscore

In [6]:
def quiesce( alpha, beta ):
    stand_pat = evaluate_board()
    if( stand_pat >= beta ):
        return beta
    if( alpha < stand_pat ):
        alpha = stand_pat

    for move in board.legal_moves:
        if board.is_capture(move):
            board.push(move)        
            score = -quiesce( -beta, -alpha )
            board.pop()

            if( score >= beta ):
                return beta
            if( score > alpha ):
                alpha = score  
    return alpha

In [7]:
def minimax_move(depth):
    bestMove = chess.Move.null()
    bestValue = -99999
    alpha = -100000
    beta = 100000
    for move in board.legal_moves:
        board.push(move)
        boardValue = -alphabeta(-beta, -alpha, depth-1)
        if boardValue > bestValue:
            bestValue = boardValue;
            bestMove = move
        if( boardValue > alpha ):
            alpha = boardValue
        board.pop()
    return bestMove

# Playing with pretrained Leela Chess (Alpha Zero)

In [8]:
import os, sys
sys.path.append(os.path.join('./chess-alpha-zero', 'src'))

from chess_zero.agent.player_chess import ChessPlayer
from chess_zero.config import Config, PlayWithHumanConfig
from chess_zero.env.chess_env import ChessEnv

from chess_zero.agent.model_chess import ChessModel
from chess_zero.lib.model_helper import load_best_model_weight

import tensorflow.compat.v1 as tf


def get_player(config):
    from chess_zero.agent.model_chess import ChessModel
    from chess_zero.lib.model_helper import load_best_model_weight
    model = ChessModel(config)
    if not load_best_model_weight(model):
        raise RuntimeError("Best model not found!")
    return ChessPlayer(config, model.get_pipes(config.play.search_threads))

default_config = Config()
PlayWithHumanConfig().update_play_config(default_config.play)
me_player = get_player(default_config)
env = ChessEnv().reset()
board = env.board

Using TensorFlow backend.




Instructions for updating:
keep_dims is deprecated, use keepdims instead

Instructions for updating:
keep_dims is deprecated, use keepdims instead







In [9]:
def minimax_vs_leela(minimax_depth=2, print_board=False):
    isMinimaxTurn = True
    minimax_total_steps = 0
    leela_total_steps = 0
    
    if print_board:
        display(SVG(chess.svg.board(board=env.board,size=250)))

    while True:
        if isMinimaxTurn:
            start = timeit.default_timer()
            action = minimax_move(minimax_depth)
            duration = timeit.default_timer()-start

            env.step(f'{action}')
            minimax_total_steps += 1
            isMinimaxTurn = False

            print(f'Minimax (white) action: {action} in {duration:.3f} sec')
        else:
            start = timeit.default_timer()
            action = me_player.action(env, False)
            duration = timeit.default_timer()-start

            env.step(action)
            leela_total_steps += 1
            isMinimaxTurn = True

            print(f'Leela (black) action: {action} in {duration:.3f} sec')


        board = env.board
        
        if print_board:
            display(SVG(chess.svg.board(board=env.board,size=250)))

        if env.done: 
            winner = 'minimax' if env.white_won else 'leela'
            winner_steps = minimax_total_steps if env.white_won else leela_total_steps
            print(f"{winner} won in {winner_steps} steps \n \n" )
            
            return [winner, winner_steps]
            break;

In [10]:
env = ChessEnv().reset()
board = env.board
minimax_vs_leela()

Minimax (white) action: g1f3 in 0.019 sec
Leela (black) action: c7c5 in 17.337 sec
Minimax (white) action: d2d4 in 0.044 sec
Leela (black) action: c5d4 in 16.622 sec
Minimax (white) action: f3d4 in 0.276 sec
Leela (black) action: e7e5 in 17.199 sec
Minimax (white) action: d4f5 in 0.139 sec
Leela (black) action: d7d5 in 16.727 sec
Minimax (white) action: d1d3 in 0.330 sec
Leela (black) action: b8c6 in 15.311 sec
Minimax (white) action: b1c3 in 0.374 sec
Leela (black) action: d5d4 in 15.928 sec
Minimax (white) action: c3b5 in 0.808 sec
Leela (black) action: c8f5 in 15.151 sec
Minimax (white) action: d3f5 in 0.350 sec
Leela (black) action: a7a6 in 15.009 sec
Minimax (white) action: c1g5 in 0.599 sec
Leela (black) action: f8e7 in 15.104 sec
Minimax (white) action: g5e7 in 2.751 sec
Leela (black) action: g8e7 in 16.335 sec
Minimax (white) action: f5g5 in 0.556 sec
Leela (black) action: e8g8 in 15.471 sec
Minimax (white) action: b5a3 in 0.914 sec
Leela (black) action: d8a5 in 13.395 sec
Mini

['leela', 47]

In [11]:
env = ChessEnv().reset()
board = env.board
minimax_vs_leela(minimax_depth=3)

Minimax (white) action: g1f3 in 0.107 sec
Leela (black) action: c7c5 in 16.991 sec
Minimax (white) action: b1c3 in 0.529 sec
Leela (black) action: e7e6 in 17.964 sec
Minimax (white) action: e2e4 in 0.851 sec
Leela (black) action: a7a6 in 18.663 sec
Minimax (white) action: f1c4 in 3.625 sec
Leela (black) action: b7b5 in 16.661 sec
Minimax (white) action: c4d3 in 7.601 sec
Leela (black) action: c8b7 in 15.463 sec
Minimax (white) action: e1g1 in 25.066 sec
Leela (black) action: c5c4 in 14.582 sec
Minimax (white) action: d3e2 in 402.351 sec
Leela (black) action: g8f6 in 15.435 sec
Minimax (white) action: e4e5 in 1736.129 sec
Leela (black) action: f6d5 in 16.072 sec
Minimax (white) action: c3d5 in 114.580 sec
Leela (black) action: b7d5 in 15.864 sec
Minimax (white) action: d2d4 in 64.650 sec
Leela (black) action: b8c6 in 14.787 sec
Minimax (white) action: c1g5 in 51.897 sec
Leela (black) action: f8e7 in 16.027 sec
Minimax (white) action: g5e7 in 23.902 sec
Leela (black) action: d8e7 in 16.0

['leela', 55]